In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!pip install mlflow torch torchvision timm --quiet

In [26]:
import mlflow

MLFLOW_DIR = "/content/drive/MyDrive/deepfake-detection/runs"
mlflow.set_tracking_uri(f"file:{MLFLOW_DIR}")

In [27]:
import os
import mlflow
import mlflow.pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, recall_score, precision_score
import numpy as np
from PIL import Image
import random
import timm
import matplotlib.pyplot as plt
import seaborn as sns

# MLFlow Setup


In [28]:
config = {
    "experiment_name": "efficientnet_b1_deepfake_binary_large_random",
    "model_name": "efficientnet_b1_custom",
    "data_path": "/content/drive/MyDrive/deepfake-detection/datasets/large_dataset_random",
    "batch_size": 32,
    "num_epochs": 12,
    "learning_rate": 1e-4,
    "img_height": 218,
    "img_width": 178,
    "dropout": 0.8,
    "optimizer": "adagrad",
    "loss_fn": "bcewithlogits",
    "random_seed": 42
}

In [29]:
mlflow.set_tracking_uri("file:/content/drive/MyDrive/deepfake-detection/runs")
mlflow.set_experiment(config["experiment_name"])

<Experiment: artifact_location='file:///content/drive/MyDrive/deepfake-detection/runs/737258101892627435', creation_time=1748855083682, experiment_id='737258101892627435', last_update_time=1748855083682, lifecycle_stage='active', name='efficientnet_b1_deepfake_binary_large_random', tags={}>

# Data Preparation

In [30]:
class RealVsFakeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        for subdir in os.listdir(root_dir):
            subdir_path = os.path.join(root_dir, subdir)
            if not os.path.isdir(subdir_path):
                continue
            label = 0 if subdir.lower() == "real" else 1
            for fname in os.listdir(subdir_path):
                if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                    self.samples.append((os.path.join(subdir_path, fname), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


In [31]:
torch.manual_seed(config["random_seed"])

transform = transforms.Compose([
    transforms.Resize((config["img_height"], config["img_width"])),
    transforms.ToTensor()
])

dataset = RealVsFakeDataset(config["data_path"], transform=transform)

total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(config["random_seed"])
)

train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=2)

# Model Definition

In [32]:
class EfficientNetB1Custom(nn.Module):
  ### Porcile Paper Approach
  # Freeze Backbone, Adapt to binary classification
    def __init__(self, dropout=0.8):
        super().__init__()
        self.backbone = timm.create_model('efficientnet_b1', pretrained=True)
        for param in self.backbone.parameters():
            param.requires_grad = False  # Freeze backbone
        in_features = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Identity()
        self.head = nn.Sequential(
            nn.Linear(in_features, 2048),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(2048, 1)
        )
    def forward(self, x):
        feats = self.backbone(x)
        out = self.head(feats)
        return out.squeeze(1)

In [33]:
model = EfficientNetB1Custom(dropout=config["dropout"])

In [34]:
def train_one_epoch(model, loader, optimizer, loss_fn, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    all_labels, all_outputs = [], []
    for images, labels in loader:
        images, labels = images.to(device), labels.float().to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        preds = torch.sigmoid(outputs) > 0.5
        correct += (preds == labels.bool()).sum().item()
        total += labels.size(0)
        all_labels.extend(labels.cpu().numpy())
        all_outputs.extend(torch.sigmoid(outputs).detach().cpu().numpy())
    acc = correct / total
    avg_loss = running_loss / total
    return avg_loss, acc, np.array(all_labels), np.array(all_outputs)

In [35]:
def evaluate(model, loader, loss_fn, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    all_labels, all_outputs = [], []
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.float().to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            running_loss += loss.item() * images.size(0)
            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels.bool()).sum().item()
            total += labels.size(0)
            all_labels.extend(labels.cpu().numpy())
            all_outputs.extend(torch.sigmoid(outputs).detach().cpu().numpy())
    acc = correct / total
    avg_loss = running_loss / total
    return avg_loss, acc, np.array(all_labels), np.array(all_outputs)

In [36]:
def plot_and_log_curve(train_values, val_values, ylabel, fname):
    import matplotlib.pyplot as plt
    plt.figure()
    plt.plot(train_values, label='Train')
    plt.plot(val_values, label='Validation')
    plt.xlabel('Epoch')
    plt.ylabel(ylabel)
    plt.title(f'{ylabel} Curve')
    plt.legend()
    plt.grid(True)
    plt.savefig(fname)
    mlflow.log_artifact(fname)
    plt.close()

In [37]:
def plot_and_log_confusion_matrix(y_true, y_pred, step, label="val"):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(4,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(f"{label.capitalize()} Confusion Matrix")
    fname = f"{label}_confusion_matrix_{step}.png"
    plt.savefig(fname)
    mlflow.log_artifact(fname)
    plt.close()

In [38]:
import os
import shutil
import numpy as np
import warnings
from sklearn.exceptions import UndefinedMetricWarning

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientNetB1Custom(dropout=config["dropout"]).to(device)
loss_fn = nn.BCEWithLogitsLoss()

if config["optimizer"].lower() == "adagrad":
    optimizer = optim.Adagrad(model.parameters(), lr=config["learning_rate"])
else:
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])

best_val_f1 = 0
best_model_path = "/tmp/best_efficientnet_b1.pth"

with mlflow.start_run():
    mlflow.log_params(config)

    train_loss_list, train_acc_list = [], []
    val_loss_list, val_acc_list = [], []

    for epoch in range(config["num_epochs"]):
        train_loss, train_acc, train_labels, train_outputs = train_one_epoch(
            model, train_loader, optimizer, loss_fn, device)
        val_loss, val_acc, val_labels, val_outputs = evaluate(
            model, val_loader, loss_fn, device)
        val_labels = np.array(val_labels).astype(int).flatten()

        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        val_loss_list.append(val_loss)
        val_acc_list.append(val_acc)

        val_preds = (val_outputs > 0.5).astype(int)
        val_preds = np.array(val_preds).astype(int).flatten()

        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UndefinedMetricWarning)
            if len(np.unique(val_labels)) < 2:
                val_f1 = 0.0
                val_tpr = 0.0
                val_fpr = 0.0
                val_auc = float('nan')
            else:
                val_f1 = f1_score(val_labels, val_preds, zero_division=0)
                val_tpr = recall_score(val_labels, val_preds, zero_division=0)
                val_fpr = 1 - precision_score(1-val_labels, 1-val_preds, zero_division=0)
                try:
                    val_auc = roc_auc_score(val_labels, val_outputs)
                except:
                    val_auc = float('nan')

        mlflow.log_metrics({
            "train_loss": train_loss, "train_acc": train_acc,
            "val_loss": val_loss, "val_acc": val_acc,
            "val_f1": val_f1, "val_tpr": val_tpr, "val_auc": val_auc,
        }, step=epoch)

        print(f"Epoch {epoch+1}/{config['num_epochs']}: "
              f"train_loss={train_loss:.4f}, train_acc={train_acc:.4f}, "
              f"val_loss={val_loss:.4f}, val_acc={val_acc:.4f}, "
              f"val_f1={val_f1:.4f}, val_auc={val_auc:.4f}")

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), best_model_path)
            mlflow.log_artifact(best_model_path)

    plot_and_log_curve(train_loss_list, val_loss_list, "Loss", "loss_curve.png")
    plot_and_log_curve(train_acc_list, val_acc_list, "Accuracy", "accuracy_curve.png")

    # Save the full model for reproducibility (architecture + weights)
    final_model_path = "/tmp/final_model"
    mlflow.pytorch.save_model(model, final_model_path)
    mlflow.log_artifacts(final_model_path, artifact_path="final_model")
    shutil.rmtree(final_model_path)

    print("Training finished. Best validation F1:", best_val_f1)
    plot_and_log_confusion_matrix(val_labels, val_preds, step="final", label="val")


Epoch 1/12: train_loss=0.6339, train_acc=0.7106, val_loss=0.5761, val_acc=0.9580, val_f1=0.9554, val_auc=0.9990
Epoch 2/12: train_loss=0.5450, train_acc=0.8652, val_loss=0.4807, val_acc=0.9729, val_f1=0.9718, val_auc=0.9995
Epoch 3/12: train_loss=0.4552, train_acc=0.9316, val_loss=0.3790, val_acc=0.9886, val_f1=0.9884, val_auc=0.9996
Epoch 4/12: train_loss=0.3725, train_acc=0.9562, val_loss=0.2890, val_acc=0.9922, val_f1=0.9920, val_auc=0.9998
Epoch 5/12: train_loss=0.3033, train_acc=0.9629, val_loss=0.2261, val_acc=0.9915, val_f1=0.9913, val_auc=0.9998
Epoch 6/12: train_loss=0.2486, train_acc=0.9704, val_loss=0.1783, val_acc=0.9915, val_f1=0.9913, val_auc=0.9999
Epoch 7/12: train_loss=0.2057, train_acc=0.9736, val_loss=0.1392, val_acc=0.9929, val_f1=0.9928, val_auc=0.9998
Epoch 8/12: train_loss=0.1705, train_acc=0.9808, val_loss=0.1149, val_acc=0.9915, val_f1=0.9913, val_auc=0.9998
Epoch 9/12: train_loss=0.1524, train_acc=0.9791, val_loss=0.0965, val_acc=0.9943, val_f1=0.9942, val_auc

2025/06/02 11:13:52 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Training finished. Best validation F1: 0.9942196531791907


In [39]:
model.load_state_dict(torch.load(best_model_path))
model.eval()

test_labels, test_outputs = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        test_outputs.extend(torch.sigmoid(outputs).cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

test_labels = np.array(test_labels).astype(int).flatten()
test_preds = (np.array(test_outputs) > 0.5).astype(int).flatten()

In [40]:
from sklearn.metrics import f1_score, recall_score, precision_score, roc_auc_score

test_f1 = f1_score(test_labels, test_preds, zero_division=0)
test_recall = recall_score(test_labels, test_preds, zero_division=0)
test_precision = precision_score(test_labels, test_preds, zero_division=0)
test_auc = roc_auc_score(test_labels, np.array(test_outputs)) if len(np.unique(test_labels)) > 1 else float('nan')

print(f"Test F1: {test_f1:.4f}, Test Recall: {test_recall:.4f}, Test Precision: {test_precision:.4f}, Test AUC: {test_auc:.4f}")

Test F1: 0.9924, Test Recall: 0.9909, Test Precision: 0.9939, Test AUC: 0.9998


In [41]:
mlflow.log_metrics({
    "test_f1": test_f1,
    "test_recall": test_recall,
    "test_precision": test_precision,
    "test_auc": test_auc
})


plot_and_log_confusion_matrix(test_labels, test_preds, step="final", label="test")